### Clone repository and install libraries

In [ ]:
import os
os.chdir("/content/")
!rm -rf ASR_Conformer

In [ ]:
#Going inside folder ASR_Conformer
import os
os.chdir("ASR_Conformer")

In [ ]:
!pip install uv
!uv sync

In [ ]:
#to create a run that shows augmentations
# !uv run scripts/show_augs.py

### Download checkpoints and dataset

In [ ]:
#downloading all checkpoints and example dataset with this command
# !uv run scripts/download_gdrive.py

In [11]:
#Or specify your own links like in the file and call function for downloading
from scripts.download_gdrive import download_dataset, download_models
GDRIVE_URLS = {
    "models": {
        "https://drive.google.com/uc?id=1a1gjSXB3mMsNOHcdndhJW6ABZ8HzBEFm": "data/models/chk_bpeablation.pth",  # bpeablation
        "https://drive.google.com/uc?id=1yPsuK5BZahriXlwl87TqY6WUuxO5WSH1": "data/models/chk_train-other-500.pth"   # train-other-500
    },
    "dataset": {# YOUR LINK HERE FOR DATASET
        "https://drive.google.com/uc?id=1ZEqEX6s7lWvCqBRclKyRW4TwJQFcxH4F": "data/datasets/with_gt", 
        "https://drive.google.com/uc?id=1rEynDEqEaYSL-2tI7dLTuXAPzLDCl-VY": "data/datasets/without_gt"
    }
}
download_models(GDRIVE_URLS)
download_dataset(GDRIVE_URLS)

/home/torfinhell/Ml/DSP/ASR_Conformer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading...
From (original): https://drive.google.com/uc?id=1a1gjSXB3mMsNOHcdndhJW6ABZ8HzBEFm
From (redirected): https://drive.google.com/uc?id=1a1gjSXB3mMsNOHcdndhJW6ABZ8HzBEFm&confirm=t&uuid=ea578f82-512c-48ea-b90a-5d0c7137993d
To: /home/torfinhell/Ml/DSP/ASR_Conformer/data/models/chk_bpeablation.pth
100%|██████████| 114M/114M [00:12<00:00, 8.85MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1yPsuK5BZahriXlwl87TqY6WUuxO5WSH1
From (redirected): https://drive.google.com/uc?id=1yPsuK5BZahriXlwl87TqY6WUuxO5WSH1&confirm=t&uuid=61ce747f-d611-4e33-b186-faec36966468
To: /home/torfinhell/Ml/DSP/ASR_Conformer/data/models/chk_train-other-500.pth
100%|██████████| 114M/114M [00:11<00:00, 9.72MB/s] 


### Inference

##### inference on test_other librispeech dataset

In [1]:
#Inferences model on test-other  dataset and produces metrics
# Also train-clean-100 is downloaded to create the bpe vocabulary

from pathlib import Path
model_dir = "data/models"
output_dir="test_other_res"

for model_path in Path(model_dir).glob("*.pth"):
    print(f"{model_path} inferenced in :\n")
    if(str(model_path)=="data/models/chk_bpeablation.pth"):
        !uv run inference.py \
            inferencer.from_pretrained={model_path} text_encoder=CTCEncoder \
            inferencer.save_path={output_dir} text_encoder.beam_size=100 \
            -cn=inference_all_metrics
    else:
        !uv run inference.py \
            inferencer.from_pretrained={model_path} text_encoder=BpeEncoder \
            inferencer.save_path={output_dir} text_encoder.beam_size=100 \
            -cn=inference_all_metrics


data/models/chk_bpeablation.pth inferenced in :

Conformer(
  (conv_subsampling): Conv2dSubsampling(
    (subsampling): Sequential(
      (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2))
      (1): ReLU()
      (2): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2))
      (3): ReLU()
    )
  )
  (linear1): Linear(in_features=19, out_features=144, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (conformer_blocks): Sequential(
    (0): ConformerBlock(
      (ffn1): FeedForwardNet(
        (sequential): Sequential(
          (0): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
          (1): Linear(in_features=144, out_features=576, bias=True)
          (2): SwishActivation()
          (3): Dropout(p=0.1, inplace=False)
          (4): Linear(in_features=576, out_features=144, bias=True)
          (5): Dropout(p=0.1, inplace=False)
        )
      )
      (mhsa): MultiHeadSelfAttention(
        (layer_norm): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
        (pos_

##### inference on dataset dir with and without transcrptions

In [22]:
#this way we inference our models on dataset_dir
model_dir = "data/models"
from pathlib import Path
#choose type with gt or without and specify your dataset_dir, dataset_dir doesnt have to depend on gt_name
gt_name="with_gt"
# gt_name="without_gt" #metrics will return 0.0
dataset_dir = f"data/datasets/{gt_name}/test_data"
for model_path in Path(model_dir).glob("*.pth"):
    print(f"{model_path} inferenced in :\n")
    if(str(model_path)=="data/models/chk_bpeablation.pth"):
        !uv run inference.py dataloader=onebatchtest \
            inferencer.dataset_dir={dataset_dir} \
            inferencer.from_pretrained={model_path} \
            inferencer.save_path={gt_name} \
            text_encoder=CTCEncoder -cn=inference
    else:
        !uv run inference.py dataloader=onebatchtest \
            inferencer.dataset_dir={dataset_dir} \
            inferencer.from_pretrained={model_path} \
            inferencer.save_path={gt_name} \
            text_encoder=BpeEncoder -cn=inference

data/models/chk_bpeablation.pth inferenced in :

Conformer(
  (conv_subsampling): Conv2dSubsampling(
    (subsampling): Sequential(
      (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2))
      (1): ReLU()
      (2): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2))
      (3): ReLU()
    )
  )
  (linear1): Linear(in_features=19, out_features=144, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (conformer_blocks): Sequential(
    (0): ConformerBlock(
      (ffn1): FeedForwardNet(
        (sequential): Sequential(
          (0): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
          (1): Linear(in_features=144, out_features=576, bias=True)
          (2): SwishActivation()
          (3): Dropout(p=0.1, inplace=False)
          (4): Linear(in_features=576, out_features=144, bias=True)
          (5): Dropout(p=0.1, inplace=False)
        )
      )
      (mhsa): MultiHeadSelfAttention(
        (layer_norm): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
        (pos_

### Calculate Metrics

In [ ]:
#produces cer and wer metrics for two folders with preds and gt texts
prediction_path=f"data/saved/{gt_name}"
ground_truth_path=f"data/datasets/{gt_name}/test_data/transcriptions"
!uv run scripts/calculate_wer_cer.py prediction_path={prediction_path} ground_truth_path={ground_truth_path}

CER Mean is 0.02988238907058401 and WER Mean is 0.09003463203463202
